In [81]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import folium

In [82]:
#Import Dataset
poiedin = pd.read_csv("data-ijcai15/poiList-ijcai15/POI-Edin.csv", sep =";")
costprofcat = pd.read_csv("C:/Users/MyBook Hype AMD/Project/tubesvdi/data-ijcai15/costProf-ijcai15/costProfCat-EdinPOI-all.csv", sep = ";")
uservisit = pd.read_csv("C:/Users/MyBook Hype AMD/Project/tubesvdi/data-ijcai15/userVisits-ijcai15/userVisits-Edin.csv", sep = ";")

poiedin

,poiID,poiName,lat,long,theme
0,1,Edinburgh_Castle,55.948611,-3.200833,Historical
1,2,Holyrood_Palace,55.952500,-3.172500,Historical
2,3,National_Museum_of_Scotland,55.946940,-3.190000,Museum
3,4,Scottish_Parliament_Building,55.952158,-3.175204,Structure
4,5,Forth_Rail_Bridge,56.000421,-3.388726,Structure
5,6,Forth_Road_Bridge,56.000830,-3.404170,Structure
6,7,Arthur%27s_Seat,55.944170,-3.161940,Park
7,8,Scott_Monument,55.952417,-3.193278,Historical
8,9,St_Giles%27_Cathedral,55.949440,-3.190830,Cultural
9,10,Calton_Hill,55.955280,-3.182220,Park


## Preprocessing Data

Merging data poiedin sama costprofcat supaya lebih jelas dari satu destinasi wisata ke wisata lainnya dengan biaya dan populeritasnya

In [87]:
# Merge df2 with df1 to get 'from' location details
merged_df = costprofcat.merge(poiedin, left_on="from", right_on="poiID", suffixes=('', '_from'))

# Merge again with df1 to get 'to' location details
merged_df = merged_df.merge(poiedin, left_on="to", right_on="poiID", suffixes=('_from', '_to'))

# Create the new 'name' column
merged_df["name"] = merged_df["poiName_from"] + " - " + merged_df["poiName_to"]

# Select relevant columns for the final DataFrame
final_df = merged_df[[
    "name", "cost", "profit", "category", "lat_from", "long_from", "lat_to", "long_to"
]]

# Rename latitude and longitude columns for clarity
final_df = final_df.rename(columns={
    "lat_from": "from_lat",
    "long_from": "from_long",
    "lat_to": "to_lat",
    "long_to": "to_long"
})

# Display the final DataFrame
print(final_df)


                                                  name          cost  profit  \
0                   Edinburgh_Castle - Holyrood_Palace   1818.249473     699   
1        National_Museum_of_Scotland - Holyrood_Palace   1254.142563     699   
2       Scottish_Parliament_Building - Holyrood_Palace    172.775758     699   
3                  Forth_Rail_Bridge - Holyrood_Palace  14486.057581     699   
4                  Forth_Road_Bridge - Holyrood_Palace  15400.268955     699   
..                                                 ...           ...     ...   
751               Our_Dynamic_Earth - Edinburgh_Castle   1663.949605    4569   
752            Craiglockhart_Castle - Edinburgh_Castle   4010.288790    4569   
753  Scottish_National_Gallery_of_Modern_Art - Edin...   1684.813773    4569   
754  Old_College,_University_of_Edinburgh - Edinbur...    860.971474    4569   
755          New_College,_Edinburgh - Edinburgh_Castle    375.229158    4569   

       category   from_lat  from_long  

In [88]:
final_df.head()

,name,cost,profit,category,from_lat,from_long,to_lat,to_long
0,Edinburgh_Castle - Holyrood_Palace,1818.249473,699,Historical,55.948611,-3.200833,55.9525,-3.1725
1,National_Museum_of_Scotland - Holyrood_Palace,1254.142563,699,Historical,55.946940,-3.190000,55.9525,-3.1725
2,Scottish_Parliament_Building - Holyrood_Palace,172.775758,699,Historical,55.952158,-3.175204,55.9525,-3.1725
3,Forth_Rail_Bridge - Holyrood_Palace,14486.057581,699,Historical,56.000421,-3.388726,55.9525,-3.1725
4,Forth_Road_Bridge - Holyrood_Palace,15400.268955,699,Historical,56.000830,-3.404170,55.9525,-3.1725


In [86]:
edinburgh_map = folium.Map(location=[55.9533, -3.1883], zoom_start=13)

# Color mapping for categories
category_colors = {
    "Historical": "blue",
    "Museum": "green",
    "Structure": "orange",
    "Park": "purple",
    "Cultural": "red",
    "Entertainment": "pink",
}

for _, row in poiedin.iterrows():
    folium.Marker(
        location=[row["lat"], row["long"]],  # Access latitude and longitude from DataFrame
        popup=f"<b>{row['poiName']}</b><br>Category: {row['theme']}",
        icon=folium.Icon(color=category_colors.get(row["theme"], "gray"))  # Use category for color
    ).add_to(edinburgh_map)

# Save the map to an HTML file
edinburgh_map.save("edinburgh_poi_map.html")